# Exemple d'appel à l'API prédicat

Created on Tue Jan 19 09:29:03 2021

@author: Yves-Laurent Benichou et Jérémy L'Hour

In [1]:
import requests
import pandas as pd
url='https://api.lab.sspcloud.fr/predicat/' # change with url of the service API

In [2]:
class predicat:
    """
    predicat:
        class to call the PREDICAT API.
    """
    def __init__(self):
        """
        init with a requests.session()
        
        
        """
        self.session = requests.session()
        self.url = url
        
    def predict(self, libelle, k=1, verbose=False, trace=False):
        """
        predict:
            call the API to predict the items in libelle

        @param k (int): number of categories to predict
        @param verbose (Bool): return also the label of the predicted category
        @param libelle (list of str): list of items to predict 
        """
        if type(libelle) == str:
            libelle = [libelle]
        
        call_url = self.url+'label?k='+str(k)+'&v='+str(verbose*1)+'&q='+'&q='.join(libelle)
        response = self.session.get(call_url)
        
        if response.ok and trace:
            print('Response received.')
        return response.json()
    
    def decode(self, categorie, trace=False):
        """
        decode:
            returns the label of a category

        @param categorie (list): categories to decode
        """
        if type(categorie) == str:
            categorie = [categorie]
            
        call_url = self.url+'label_description?q='+'&q='.join(categorie)
        response = self.session.get(call_url)
        if response.ok and trace:
            print('Response received.')
        return response.json()
    
classifier = predicat()   

In [3]:
# Une seule prédiction pour un libelle

libelle = 'Compote de Pommes'
prediction = classifier.predict([libelle])
print(prediction)

{'Compote de Pommes': [{'label': '01.1.8.2.1', 'proba': '1.000', 'confiance': '0.985'}]}


In [4]:
# Retrouver l'intitulé de la categorie
label = prediction[libelle][0]['label']
dico = classifier.decode([label])
print('Le code '+label+' correspond à '+dico[label])

Le code 01.1.8.2.1 correspond à Confitures, compotes et miel


In [5]:
# Plusieurs prédictions pour un libelle

libelle = 'Compote de Pommes'
predictions = classifier.predict([libelle], k=5)
print(predictions)

{'Compote de Pommes': [{'label': '01.1.8.2.1', 'proba': '1.000', 'confiance': '0.985'}, {'label': '01.1.1.4.1', 'proba': '0.015', 'confiance': None}, {'label': '01.1.3.4.1', 'proba': '0.001', 'confiance': None}, {'label': '01.1.9.1.1', 'proba': '0.000', 'confiance': None}, {'label': '02.2.0.2.1', 'proba': '0.000', 'confiance': None}]}


In [6]:
# Affichage du top-k
for i, item in enumerate(predictions[libelle]):
    print('Prédiction '+str(i+1)+': '+item['label']+' avec probabilité '+item['proba'])

Prédiction 1: 01.1.8.2.1 avec probabilité 1.000
Prédiction 2: 01.1.1.4.1 avec probabilité 0.015
Prédiction 3: 01.1.3.4.1 avec probabilité 0.001
Prédiction 4: 01.1.9.1.1 avec probabilité 0.000
Prédiction 5: 02.2.0.2.1 avec probabilité 0.000


In [7]:
# Une prédiction pour plusieurs libelles et mise dans un pd.DataFrame()

libelles = ['Compote de pommes', 'Yaourts 0% x6', 'Biscuits chocolat x2']

predictions = classifier.predict(libelles)
row_list = [predictions[item][0] for item in libelles]

df = pd.DataFrame(row_list)
df['libelle'] = libelles
df.head()

,label,proba,confiance,libelle
0,01.1.8.2.1,1.000,0.985,Compote de pommes
1,01.1.4.4.1,1.000,0.999,Yaourts 0% x6
2,01.1.1.4.5,0.998,0.975,Biscuits chocolat x2


In [8]:
# A partir d'un csv

def predict_from_csv(file):
    """
    predict_from_csv:
        inputs a csv file and outputs the same file with three extra columns (label, proba, confiance)
        
    @param file (str): a csv file
    @param session (): a requests.session() object
    """
    with open(file) as f:
        df = pd.read_csv(f)
        
    predictions = classifier.predict(df['libelle'].to_list())
    row_list = [predictions[item][0] for item in df['libelle'].to_list()]

    final_df = pd.DataFrame(row_list)
    final_df['libelle'] = df['libelle']
    final_df = final_df[['libelle', 'label', 'proba', 'confiance']]
    final_df.to_csv('output.csv')
    return final_df


In [9]:
final_df = predict_from_csv('libelles.csv')
final_df.head()

,libelle,label,proba,confiance
0,Compote abricot,01.1.8.2.1,1.000,0.999
1,Yaourt Fruits Yoplait,01.1.4.4.1,1.000,0.997
2,Jus d'orange,01.1.1.4.1,0.893,0.308
3,Oranges 500g,01.1.8.2.1,0.615,0.245
4,Gnocchi Lustucru,01.1.1.6.1,1.000,0.943


In [11]:
dico = classifier.decode(final_df['label'].to_list())
final_df['intitule'] = final_df['label'].map(dico)
final_df

,libelle,label,proba,confiance,intitule
0,Compote abricot,01.1.8.2.1,1.000,0.999,"Confitures, compotes et miel"
1,Yaourt Fruits Yoplait,01.1.4.4.1,1.000,0.997,Yaourt
2,Jus d'orange,01.1.1.4.1,0.893,0.308,Pâtisserie fraîche
3,Oranges 500g,01.1.8.2.1,0.615,0.245,"Confitures, compotes et miel"
4,Gnocchi Lustucru,01.1.1.6.1,1.000,0.943,Pâtes alimentaires et couscous
5,T-Shirt HOMME XXL,03.1.2.2.2,0.438,0.370,Tailleurs et ensembles pour femme
6,Robe ete taille M,03.1.2.2.2,0.400,0.372,Tailleurs et ensembles pour femme
7,Baskets 41 H,03.2.1.2.1,1.000,0.852,Chaussures de ville pour femme
8,Pates barilla,01.1.1.6.1,1.000,0.999,Pâtes alimentaires et couscous
9,biberon tetine silicone,12.3.2.2.1,1.000,0.989,Articles de puériculture
